In [57]:
from collections import namedtuple
from enum import Enum

In [58]:
Condition = Enum("Condition", ("CURE", "HEALTHY", "SICK", "DYING", "DEAD"))
Agent = namedtuple("Agent", ("name", "category"))

In [ ]:
def meetup(agent_listing: tuple) -> list:
    """Model the outcome of the meetings of pairs of agents.

    The pairs of agents are ((a[0], a[1]), (a[2], a[3]), ...). If there's an uneven
    number of agents, the last agent will remain the same.

    Notes
    -----
    The rules governing the meetings were described in the question. The outgoing
    listing may change its internal ordering relative to the incoming one.

    Parameters
    ----------
    agent_listing : tuple of Agent
        A listing (tuple in this case) in which each element is of the Agent
        type, containing a 'name' field and a 'category' field, with 'category' being
        of the type Condition.

    Returns
    -------
    updated_listing : list
        A list of Agents with their 'category' field changed according to the result
        of the meeting.
    """
    update_listing = [] 

    # handle healthy and dead agents 
    healthy_and_dead = [agent for agent in agent_listing
                        if agent.category == Condition.HEALTHY or agent.category == Condition.DEAD]
    
    # add them as is to the result
    update_listing.extend(healthy_and_dead) 

    # list the agents that will change after meetings
    sick_and_cure = [agent for agent in agent_listing
                        if agent.category != Condition.HEALTHY and agent.category != Condition.DEAD]

    print(f"healthy and dead is {healthy_and_dead} \n origin list is {agent_listing} \n after removing {sick_and_cure} ")

    

In [60]:
data1 = (Agent("Buddy", Condition.CURE), Agent("Holly", Condition.DEAD))

In [61]:
meetup(data1)

healthy and dead is [Agent(name='Holly', category=<Condition.DEAD: 5>)] 
 origin list is (Agent(name='Buddy', category=<Condition.CURE: 1>), Agent(name='Holly', category=<Condition.DEAD: 5>)) 
 after removing [Agent(name='Buddy', category=<Condition.CURE: 1>)] 
